In [ ]:
from datetime import datetime as dt
from aiohttp import ClientSession as UA, TCPConnector
from datetime import datetime as dt, timedelta as td
from finam_csv_loader import download_tick_verbose
from asyncio import sleep
import pandas as pd
import io


ticker_code, ticker_id = 'TATN', 825

ua = UA(connector=TCPConnector(limit=5))
cols = [ 'time','last','vol', 'oper' ]
df = None  # pd.DataFrame([], columns=cols)
d = dt.now().date()
start = dt(2019, 7, 1).date()
end = dt(2019, 8, 1).date()
diff = 0

while True:
    d = start + td(days=diff)
    if not d < end:
        break
    else:
        print(d)
        diff += 1

    file = await download_tick_verbose(ua, ticker_id, ticker_code, d)
    _df = pd.read_csv(io.StringIO(file),
                      header=0, sep=';', decimal='.', encoding='cp1251',
                      usecols=[1,2,3,5],
                      names=cols,)

    _df['time'] = _df['time'].apply(lambda t: f"{str(d)} {t}")
    _df['time'] = pd.to_datetime(_df['time'], format='%Y-%m-%d %H:%M:%S')

    if df is None:
        df = _df.copy()
    else:
        df = df.append(_df, ignore_index=True)

    await sleep(1.)

await ua.close()

df['vol'] = df['vol'].astype(float)
df['oper'] = df['oper'].map({'B': 1, 'S': -1})

df.info()

In [33]:
import pandas as pd


store = pd.HDFStore('finam.h5', complevel=6)
# store['stock/TATN'] = df
df = store['stock/TATN']
df.head()
store.close()

In [43]:
print(df.vol.sum())
print(df['last'].min())
print(df['last'].max())
df['last'].std()

37733590.0
716.1
784.3


15.449882037519371